In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
import uuid
import json
from langchain_community.chat_message_histories.mongodb import MongoDBChatMessageHistory

from typing import Optional, List, Literal
import re
from dotenv import load_dotenv
import os
load_dotenv()

True

In [14]:
a = ProductItem()
[dict(x) for x in [a, a]]

[{'name': None, 'quantity': None}, {'name': None, 'quantity': None}]

In [2]:
data = json.load(open('chickendata.json', 'r'))
product_list = list(data.keys())
Argument = Literal['1']
Argument.__args__ = tuple(product_list)

class ProductItem(BaseModel):
    name: Optional[Argument] = Field(description="tên sản phẩm")
    quantity: Optional[int] = Field(description="số lượng muốn mua")

class Serve(BaseModel):
    # customer_name: Optional[str] = Field(description="tên của khách hàng")
    customer_phone_number: Optional[str] = Field(description="số điện thoại của khách hàng (không phải số điện thoại của Aroma)")
    order: Optional[List[ProductItem]] = Field(description="Một đơn hàng chứa danh sách các sản phẩm.")

class CheckInventory(BaseModel):
    order: List[ProductItem] = Field(description="Một đơn hàng chứa danh sách các sản phẩm.")

@tool("serve", args_schema=Serve,
      # return_direct=True #Will return the direct result in string form
      )
def serve(customer_phone_number="", order=[]):
    """được sử dụng khi khách hàng muốn mua bánh hoặc order sản phẩm"""
    if "918815325" in customer_phone_number:
        if order!=[]:
            return "Bạn hay cung cấp đầy đủ thông tin về số điện thoại"
        return ""      
    
    if all([customer_phone_number!="", order!=[] and order!=None]):
        # return check_inventory(customer_phone_number, order)
        if re.match(r'[+]?[\d\s]{9,20}', customer_phone_number) == None:
            return "Số điện thoại không hợp lệ, vui lòng nhập lại."
        return trigger(customer_phone_number, order)
    elif all([customer_phone_number=="", order==[] or order==None]):
        return ""
    
    response = f'Bạn hay cung cấp đầy đủ thông tin về {"số điện thoại" if customer_phone_number=="" else ""}, {"đơn hàng (loại bánh + số lượng)" if order==[] else ""}.'
    return response

def check_inventory(customer_phone_number, order=[]):
    """"""
    deficient_product_list = []
    order = {x.name:x.quantity for x in order}
    print(order)
    for product in list(order.keys()):
        if product not in product_list:
            return f"sản phẩn {product} không tồn tại. xin hãy chọn một trong các sản phẩm sau (nêu kèm số lượng): {data.keys()}"
        if order[product]>data[product]:
            deficient_product_list.append(product)
    if deficient_product_list==[]:
        return trigger(customer_phone_number, order)
    else:
        return "Hiện tại:\n" + '\n'.join([f"Bánh {x} chỉ còn {data[x]} chiếc" for x in deficient_product_list]) +"\n Phiền bạn nhập lại số lượng sao cho phù hợp bạn ha ^^."


def trigger(customer_phone_number, order):
    """"""
    
    return f"Order thành công, chờ nghe điện thoại.\n gửi lại cho khách như sau:\nThông tin đơn hàng:\n số điện thoại: {customer_phone_number} \n danh sách sản phẩm: {order}"

@tool("send_menu")
def send_menu():
    """Được sử dụng khi khách hỏi về menu, thực đơn, quán có món gì, bánh gì ngon."""
    return f"Aroma xin gửi thực đơn ạ: \n {[x for x in data]}"

tools = [
    serve,
    send_menu
]

In [3]:
import re
re.match(r'[+]?[\d\s]{9,20}', '0777080706')

<re.Match object; span=(0, 10), match='0777080706'>

In [4]:
prompt = hub.pull("hwchase17/openai-tools-agent")

system_template ="""

Bạn là một người trợ lý ảo có nhiệm vụ nhận order của khách hàng cho tiệm bánh Aroma.

Một vài thông tin về tiệm bánh Aroma:
- Địa chỉ: 325a Đ. Điện Biên Phủ, Phường 15, Bình Thạnh, Thành phố Hồ Chí Minh [https://maps.app.goo.gl/6Y7jEWVKbu1Rs2Ww8]
- Số điện thoại (phone): 0918815325

Tuân thử những điều sau:
1/ Giới thiệu mình là trợ lý ảo của tiệm bánh Aroma Bakery
2/ Nếu khách hàng muốn mua bánh, sử dụng `serve` ngay lập tức.
3/ Nếu khách hàng muốn xem thực đơn, sử dụng `send_menu` ngay lập tức.
"""
prompt.messages[0] = prompt.messages[0].from_template(system_template)

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, openai_api_key = os.getenv('OPENAI_API_KEY'))

agent = create_openai_functions_agent(
    tools = tools,
    llm = llm,
    prompt = prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [5]:
def trim_messages(history_object):
    stored_messages = history_object.messages
    if len(stored_messages) <= int(os.getenv('NO_MESSAGES_REMEMBER')):
        return history_object

    history_object.clear()

    for message in stored_messages[-int(os.getenv('NO_MESSAGES_REMEMBER')):]:
        history_object.add_message(message)

    return history_object

agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: trim_messages(MongoDBChatMessageHistory(
            session_id=session_id,
            connection_string=os.getenv('MONGODB_CONNECTION_STRING'),
            database_name="MentalChatApp",
            collection_name="chat_history",
        )),
        input_messages_key="input",
        history_messages_key="chat_history",
    )

In [6]:
import uuid
session_id = uuid.uuid4().__str__()

In [7]:
agent_with_chat_history.invoke(
    {"input": 'Xin chào tôi tên là Sang\nTôi số điện thoại của Aroma là gì?\ncho tôi mua 5 crossaint và 10 bánh tiramisu'},
    config={"configurable": {"session_id": session_id}},
)

C:\Users\SANG\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_message_histories.mongodb.MongoDBChatMessageHistory` was deprecated in langchain-community 0.0.25 and will be removed in 0.2.0. An updated version of the class exists in the langchain-mongodb package and should be used instead. To use it run `pip install -U langchain-mongodb` and import as `from langchain_mongodb import MongoDBChatMessageHistory`.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `serve` with `{'customer_phone_number': '0918815325', 'order': [{'name': 'Crossaint', 'quantity': 5}, {'name': 'Tiramisu', 'quantity': 10}]}`


Xin chào Sang, số điện thoại của Aroma là 0918815325. Đơn hàng của bạn đã được ghi nhận. Cảm ơn bạn đã mua sắm tại Aroma Bakery!

> Finished chain.


{'input': 'Xin chào tôi tên là Sang\nTôi số điện thoại của Aroma là gì?\ncho tôi mua 5 crossaint và 10 bánh tiramisu',
 'chat_history': [],
 'output': 'Xin chào Sang, số điện thoại của Aroma là 0918815325. Đơn hàng của bạn đã được ghi nhận. Cảm ơn bạn đã mua sắm tại Aroma Bakery!'}

In [42]:
agent_with_chat_history.invoke(
    {"input": 'số điện thoại của Aroma'},
    config={"configurable": {"session_id": session_id}},
)



> Entering new AgentExecutor chain...

Invoking: `serve` with `{}`


Xin lỗi, có vẻ như tôi đã gây hiểu lầm. Để giúp bạn đặt hàng, vui lòng cung cấp số điện thoại của bạn và danh sách sản phẩm bạn muốn mua.

> Finished chain.


{'input': 'số điện thoại của Aroma',
 'chat_history': [HumanMessage(content='ok thực đơn đê'),
  AIMessage(content='Dưới đây là thực đơn của Aroma Bakery:\n1. Crossaint\n2. Tiramisu\n3. Bánh bò\n\nNếu bạn muốn đặt hàng, vui lòng cung cấp số điện thoại và danh sách sản phẩm bạn muốn mua.')],
 'output': 'Xin lỗi, có vẻ như tôi đã gây hiểu lầm. Để giúp bạn đặt hàng, vui lòng cung cấp số điện thoại của bạn và danh sách sản phẩm bạn muốn mua.'}

In [43]:
agent_with_chat_history.invoke(
    {"input": 'Thêm 70 Bánh Tiramisu ngon vcl và 50 bánh Crossaint.'},
    config={"configurable": {"session_id": session_id}},
)



> Entering new AgentExecutor chain...

Invoking: `serve` with `{'order': [{'name': 'Tiramisu', 'quantity': 70}, {'name': 'Crossaint', 'quantity': 50}]}`


Bạn hay cung cấp đầy đủ thông tin về số điện thoại, .Xin lỗi vì sự bất tiện này. Vui lòng cung cấp số điện thoại của bạn để chúng tôi có thể xác nhận đơn hàng của bạn.

> Finished chain.


{'input': 'Thêm 70 Bánh Tiramisu ngon vcl và 50 bánh Crossaint.',
 'chat_history': [HumanMessage(content='ok thực đơn đê'),
  AIMessage(content='Dưới đây là thực đơn của Aroma Bakery:\n1. Crossaint\n2. Tiramisu\n3. Bánh bò\n\nNếu bạn muốn đặt hàng, vui lòng cung cấp số điện thoại và danh sách sản phẩm bạn muốn mua.'),
  HumanMessage(content='số điện thoại của Aroma'),
  AIMessage(content='Xin lỗi, có vẻ như tôi đã gây hiểu lầm. Để giúp bạn đặt hàng, vui lòng cung cấp số điện thoại của bạn và danh sách sản phẩm bạn muốn mua.')],
 'output': 'Xin lỗi vì sự bất tiện này. Vui lòng cung cấp số điện thoại của bạn để chúng tôi có thể xác nhận đơn hàng của bạn.'}

In [44]:
agent_with_chat_history.invoke(
    {"input": '0777080706'},
    config={"configurable": {"session_id": session_id}},
)



> Entering new AgentExecutor chain...

Invoking: `serve` with `{'customer_phone_number': '0777080706', 'order': [{'name': 'Tiramisu', 'quantity': 70}, {'name': 'Crossaint', 'quantity': 50}]}`


Order thành công, chờ nghe điện thoại.
 gửi lại cho khách như sau:
Thông tin đơn hàng:
 số điện thoại: 0777080706 
 danh sách sản phẩm: [ProductItem(name='Tiramisu', quantity=70), ProductItem(name='Crossaint', quantity=50)]Đơn hàng của bạn đã được ghi nhận. Chúng tôi sẽ liên hệ với bạn sớm nhất có thể để xác nhận đơn hàng. Cảm ơn bạn đã mua sắm tại Aroma Bakery!

> Finished chain.


{'input': '0777080706',
 'chat_history': [AIMessage(content='Dưới đây là thực đơn của Aroma Bakery:\n1. Crossaint\n2. Tiramisu\n3. Bánh bò\n\nNếu bạn muốn đặt hàng, vui lòng cung cấp số điện thoại và danh sách sản phẩm bạn muốn mua.'),
  HumanMessage(content='số điện thoại của Aroma'),
  AIMessage(content='Xin lỗi, có vẻ như tôi đã gây hiểu lầm. Để giúp bạn đặt hàng, vui lòng cung cấp số điện thoại của bạn và danh sách sản phẩm bạn muốn mua.'),
  HumanMessage(content='Thêm 70 Bánh Tiramisu ngon vcl và 50 bánh Crossaint.'),
  AIMessage(content='Xin lỗi vì sự bất tiện này. Vui lòng cung cấp số điện thoại của bạn để chúng tôi có thể xác nhận đơn hàng của bạn.')],
 'output': 'Đơn hàng của bạn đã được ghi nhận. Chúng tôi sẽ liên hệ với bạn sớm nhất có thể để xác nhận đơn hàng. Cảm ơn bạn đã mua sắm tại Aroma Bakery!'}